In [1]:
# Main import block
import asyncio
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO

# Sneaky scrapper
import cfscrape
# Upload to google sheets
import gspread
# import df2gspread as d2g
import gspread_pandas as gsp
import numpy as np
import pandas as pd
import requests
# Upload to yandex
import yadisk
from bs4 import BeautifulSoup
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

warnings.filterwarnings("ignore")

In [2]:
##### Functions to maintain side manipulations #####
def writer(time_out=10):
    # Function for uploading dataframes into the google docs
    def google_upload():
        try:
            # Параметры подключения
            scope = [
                "https://spreadsheets.google.com/feeds",
                "https://www.googleapis.com/auth/drive",
            ]

            json_keyfile_path = "./misc/macro-parser-lme-c2f2972b48fc.json"
            excel_file_path = "./data/multiple_dataframes.xlsx"
            spreadsheet_key = "19XJClFWuwVdKx0R0Hw9owa6B6YJtgr9rgwT_0AJta4k"

            # API токен для подключения
            credentials = ServiceAccountCredentials.from_json_keyfile_name(
                json_keyfile_path, scope
            )
            gc = gspread.authorize(credentials)

            # Читаем файл
            xls = pd.ExcelFile(excel_file_path)

            # Пробегаемся по каждому листу книги и скидываем их в гугл докс
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
                d2g.upload(
                    df, spreadsheet_key, 
                    sheet_name, 
                    credentials=credentials,
                    row_names=True)
                print(f"{sheet_name} uploaded!")
                time.sleep(time_out)

            print("Uploading whole file to Google Sheets completed")

        except Exception as error:
            print(error)
            

    # Выгрузка на диск
    def yandex_upload():
        token = pd.read_csv("./misc/token.csv", header=None)
        token = token.iloc[0, 0]

        y = yadisk.YaDisk(token=token)

        try:
            y.upload("./data/multiple_dataframes.xlsx", "/macro_db/database.xlsx")

        except Exception as error:
            print(error)
            y.remove("/macro_db/database.xlsx")
            y.upload("./data/multiple_dataframes.xlsx", "/macro_db/database.xlsx")

    # Открываем все необходимые фреймы
    lme = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
    kitko = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
    lbma = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)
    nbk = pd.read_excel("./data/nbk_tenge.xlsx", index_col=0)
    shmet = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
    westmetall = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

    usd = pd.read_excel("./data/cb_currency.xlsx", sheet_name="USD", index_col=0)
    eur = pd.read_excel("./data/cb_currency.xlsx", sheet_name="EUR", index_col=0)
    ausd = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Australian_Dollar", index_col=0
    )
    yuan = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="China_Yuan", index_col=0
    )
    pound = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="British_Pound", index_col=0
    )
    tenge = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Kazakhstan_Tenge", index_col=0
    )
    yen = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Japanese_Yen", index_col=0
    )
    swiss = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Swiss_Franc", index_col=0
    )

    cb_metalls = pd.read_excel(
        "./data/cb_metalls.xlsx", sheet_name="cb_metalls", index_col=0
    )
    
    # Мержим всю собранную инфу в локальную "БД"
    with pd.ExcelWriter(
        "./data/multiple_dataframes.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
        engine="xlsxwriter",
    ) as writer:
        lme.to_excel(writer, sheet_name="LME_non_ferrous", index=False)
        kitko.to_excel(writer, sheet_name="KITCO", index=False)
        lbma.to_excel(writer, sheet_name="LBMA", index=False)
        nbk.to_excel(writer, sheet_name="nbk_tenge", index=False)
        shmet.to_excel(writer, sheet_name="SHMET", index=False)
        westmetall.to_excel(writer, sheet_name="LME_westmetall", index=False)
        usd.to_excel(writer, sheet_name="USD", index=False)
        eur.to_excel(writer, sheet_name="EUR", index=False)
        ausd.to_excel(writer, sheet_name="Australian_Dollar", index=False)
        yuan.to_excel(writer, sheet_name="China_Yuan", index=False)
        pound.to_excel(writer, sheet_name="British_Pound", index=False)
        tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge", index=False)
        yen.to_excel(writer, sheet_name="Japanese_Yen", index=False)
        swiss.to_excel(writer, sheet_name="Swiss_Franc", index=False)
        cb_metalls.to_excel(writer, sheet_name="cb_metalls", index=False)

    print("Database uploaded on yandex.disk")

    yandex_upload()
    google_upload()


# Timeout class for reattempting connection
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")

    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


# Function to convert date from json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def date_format_reverse():
    date_now = date.today()
    date_string = date_now.strftime("%Y-%m-%d %H:%M:%S")
    date_raw = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").timestamp()
    date_raw = date_raw * 1000
    return int(date_raw)


# Session creation via proxy
def get_session(url):
    # Free proxy function
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find("table").find_all("tr")  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all("td"))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == "yes":  # taking only https
                    proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # create session
    session = requests.Session()

    # random proxy
    proxy = get_free_proxies()
    counter = 0

    while counter <= len(proxy):
        try:
            with time_limit(7):
                random_proxy = proxy.sample().values[0]
                session.proxies = {"https": random_proxy}
                response = session.get(url)
                break

        except OSError:
            counter += 1
            pass

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1
            print(f"Attempt {counter+1}")

    return response

async def fetch_html(url, session):
    async with session.get(url) as response:
        return await response.text()

----

### This is the main function block ###

In [3]:
################################################################
##########################   LME ###############################
################################################################
# Асинхронная функция
async def lme_selenium_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"

    service = Service()
    driver = webdriver.Chrome(service=service)

    try:
        driver.maximize_window()
        driver.get(url=url)

        time.sleep(2)

        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )
        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )

        time.sleep(2)

        html_code = driver.page_source
        #driver.close()

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        # Приводим к тексту
        metalls_raw = data_raw[0].text

        # Убираем все лишнее
        metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
        metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # Конфигурируем новый лист по металлам
        metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]

        # Обрезаем до нужной инфы
        metalls_raw = [i[:2] for i in metalls_raw]

        # Приведем цены к числовому формату
        for metall in metalls_raw:
            metall[1] = pd.to_numeric(metall[1])

        # Формируем строку датафрейма по металлам
        dict_list = [{item[0]: item[1]} for item in metalls_raw]

        metall_df = pd.DataFrame()

        for i in dict_list:
            keys = list(i.keys())
            values = list(i.values())
            metall_df[keys[0]] = values

        # Дропаем ненужные колонки
        metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
        metall_df = metall_df.rename(
            columns={
                "LME_Aluminium": "aluminium",
                "LME_Copper": "copper",
                "LME_Lead": "lead",
                "LME_Nickel": "nickel",
                "LME_Zinc": "zink",
                "LME_Tin": "tin",
            }
        )

        # Достанем в вставим дату
        date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
        date_raw = date_raw[0]
        date_raw = str(date_raw).split(">")[1]
        date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        metall_df["date"] = date_raw

        # Поменяем порядок колонок
        new_column_order = [
            "date",
            "aluminium",
            "copper",
            "lead",
            "nickel",
            "zink",
            "tin",
        ]
        metall_df = metall_df[new_column_order]

        # Сохраняем полученные результаты
        lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
        lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

        # Дропаем дубликаты
        lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

        # Сохраняем
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_db.to_excel(writer, sheet_name="LME_main")
        
        print("LME_main is done!!!")
        display(lme_db.tail(10))

    except Exception as error:
        print(error)

    finally:
        driver.close()
        driver.quit()


In [4]:
################################################################
#########################   KITCO (main) ########################
################################################################


# Ассинхронный парсер Kitco
async def kitco_parser_async():
    async def page_getter():
        try:
            url = "https://www.kitco.com/price/fixes/london-fix"

            service = Service()
            driver = webdriver.Chrome(service=service)
            # driver.maximize_window()
            driver.get(url=url)
            time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )

            time.sleep(2)

            html_code = driver.page_source
            #driver.close()

            return html_code

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()

    html_code = await page_getter()

    try:
        # Работа над полученными данными
        soup = BeautifulSoup(html_code)

        day = soup.find_all("div", class_="grid")

        # Вычленение первичного контента
        day_content = []
        for i in range(len(day)):
            day_content.append(day[i].contents)

        # Первичное преобразование
        for container in day_content:
            for element in range(len(container)):
                try:
                    container[element] = (
                        str(container[element])
                        .replace("<div>", "")
                        .replace("</div>", "")
                        .replace("<!-- -->", "")
                    )
                    container[element]
                except:
                    pass

        # Удалим неинформативные строки
        day_content = day_content[1:]
        day_content.pop(1)

        # Преобразование дат и вычлинение значений
        for element in day_content:
            # Преобразованиие даты
            element[0] = pd.to_datetime(element[0]).strftime("%Y-%m-%d")

            # Числовые преобразования
            element[1] = pd.to_numeric(
                element[1].split("/")[1].strip().replace(",", "")
            )
            element[2] = pd.to_numeric(
                element[2].split("/")[0].strip().replace(",", "")
            )
            element[3] = pd.to_numeric(
                element[3].split("/")[1].strip().replace(",", "")
            )
            element[4] = pd.to_numeric(
                element[4].split("/")[1].strip().replace(",", "")
            )

        # Формируем датафрейм
        day_content = pd.DataFrame(
            day_content, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
        )

        # Преобразуем типы
        day_content["Date"] = pd.to_datetime(day_content["Date"])
        day_content = day_content.sort_values("Date").reset_index(drop=True)

        # Возьмем последние 5 строк
        day_update = day_content.tail()

        # Склеим с историческими данными
        historical_df = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
        historical_df = (
            pd.concat([historical_df, day_update], axis=0)
            .reset_index(drop=True)
            .drop_duplicates()
        )

        with pd.ExcelWriter(
            "./data/kitko_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            historical_df.to_excel(writer, sheet_name="kitco_metall")

        print("KITKO_main is done!!!")
        display(historical_df.tail(10))

    except Exception as e:
        print(e)

In [5]:
################################################################
#########################   LBMA (KITCO subs) ##################
################################################################


async def lbma_prescious_async():
    url_gold = "https://prices.lbma.org.uk/json/gold_pm.json"
    url_silver = "https://prices.lbma.org.uk/json/silver.json"
    url_platinum = "https://prices.lbma.org.uk/json/platinum_pm.json"
    url_paladium = "https://prices.lbma.org.uk/json/palladium_pm.json"

    async def get_raw_data(url, metall="Default"):
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # scraped_data = get_session(url=url)

        raw_data = pd.read_json(scraped_data.text)
        data = raw_data[["d", "v"]]
        data["v"] = data["v"].apply(lambda x: x[0])
        data["d"] = pd.to_datetime(data["d"])

        data.rename(columns={"d": "Date", "v": metall}, inplace=True)
        data = data.tail(4)

        return data

    try:
        gold = await get_raw_data(url_gold, metall="Gold")
        silver = await get_raw_data(url_silver, metall="Silver")
        platinum = await get_raw_data(url_platinum, metall="Platinum")
        paladium = await get_raw_data(url_paladium, metall="Palladium")

        result_df = (
            gold.merge(silver, on="Date", how="outer")
            .merge(platinum, on="Date", how="outer")
            .merge(paladium, on="Date", how="outer")
        )

        result_df.fillna(value=0, inplace=True)
        result_df = result_df.sort_values(by="Date")
        result_df.reset_index(inplace=True, drop=True)

        historical = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

        result = pd.concat([historical, result_df], axis=0).reset_index(drop=True)

        result.drop_duplicates(inplace=True)

        result = result.sort_values(by="Date")

        result.to_excel("./data/lbme_kitco_subs.xlsx", sheet_name="lbme_metall")

        print("LBMA is done!!!")
        display(result.tail(10))

    except Exception as error:
        print(error)

In [6]:
################################################################
####################   CB_currencies ###########################
################################################################


async def cb_currency():
    async def get_currency(currency):
        current_date = datetime.today().strftime("%d.%m.%Y")

        dict_of_currencies = {
            "USD": "R01235",
            "EUR": "R01239",
            "Australian_Dollar": "R01010",
            "China_Yuan": "R01375",
            "British_Pound": "R01035",
            "Kazakhstan_Tenge": "R01335",
            "Japanese_Yen": "R01820",
            "Swiss_Franc": "R01775",
        }

        url = f"https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.\
Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.\
date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={dict_of_currencies[currency]}&UniDbQuery.\
From=01.01.2023&UniDbQuery.To={current_date}"

        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        return scraped_data

    def data_reconstruction(scraped_data, name):

        # Получаем датафрейм
        data = pd.read_html(scraped_data.text.replace(",", "."))[0]
        data.columns = data.iloc[1]

        # Преобразуем
        data = data.iloc[2:]
        data = data.rename(
            columns={
                data.columns[0]: "date",
                data.columns[1]: "unit",
                data.columns[2]: "nominal",
            }
        ).reset_index(drop=True)

        # Cкидываем название индекса
        data = pd.read_csv(StringIO(data.to_string(index=False)), delim_whitespace=True)
        data["date"] = pd.to_datetime(data["date"], format="%d.%m.%Y")
        data["unit"] = pd.to_numeric(data["unit"])
        data["nominal"] = pd.to_numeric(data["nominal"])

        data = data.sort_values("date").reset_index(drop=True)

        print(f"{name} is done!")
        display(data.tail())

        return data

    try:
        raw_usd = await get_currency(currency="USD")
        raw_eur = await get_currency(currency="EUR")
        raw_ausd = await get_currency(currency="Australian_Dollar")
        raw_yuan = await get_currency(currency="China_Yuan")
        raw_pound = await get_currency(currency="British_Pound")
        raw_tenge = await get_currency(currency="Kazakhstan_Tenge")
        raw_yen = await get_currency(currency="Japanese_Yen")
        raw_swiss = await get_currency(currency="Swiss_Franc")

        usd = data_reconstruction(raw_usd, name="USD")
        eur = data_reconstruction(raw_eur, name="EUR")
        ausd = data_reconstruction(raw_ausd, name="Australian_Dollar")
        yuan = data_reconstruction(raw_yuan, name="China_Yuan")
        pound = data_reconstruction(raw_pound, name="British_Pound")
        tenge = data_reconstruction(raw_tenge, name="Kazakhstan_Tenge")
        yen = data_reconstruction(raw_yen, name="Japanese_Yen")
        swiss = data_reconstruction(raw_swiss, name="Swiss_Franc")

        with pd.ExcelWriter(
            "./data/cb_currency.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            usd.to_excel(writer, sheet_name="USD")
            eur.to_excel(writer, sheet_name="EUR")
            ausd.to_excel(writer, sheet_name="Australian_Dollar")
            yuan.to_excel(writer, sheet_name="China_Yuan")
            pound.to_excel(writer, sheet_name="British_Pound")
            tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge")
            yen.to_excel(writer, sheet_name="Japanese_Yen")
            swiss.to_excel(writer, sheet_name="Swiss_Franc")

    except Exception as error:
        print(error)
        

In [7]:
################################################################
####################   CB_metalls ##############################
################################################################


async def cb_metalls():
    try:
        # Первичные гиперпараметры
        current_year = datetime.now().year

        url = f"https://www.cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted\
=True&UniDbQuery.From=01.01.2023&UniDbQuery.To=31.12.{current_year}&UniDbQuery.Gold\
=true&UniDbQuery.Silver=true&UniDbQuery.Platinum=true&UniDbQuery.Palladium\
=true&UniDbQuery.so=1"

        # Запуск скраппера
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # Первичное получение данных
        metalls = pd.read_html(scraped_data.text.replace(",", "."))[0]

        # Переименовывание колонок
        metalls = metalls.rename(
            columns={
                metalls.columns[0]: "date",
                metalls.columns[1]: "gold",
                metalls.columns[2]: "silver",
                metalls.columns[3]: "platinum",
                metalls.columns[4]: "palladium",
            }
        )

        # Приведение данных к необходимому формату
        metalls["date"] = pd.to_datetime(metalls["date"], format="%d.%m.%Y")
        metalls["gold"] = pd.to_numeric(metalls["gold"].str.replace(" ", ""))
        metalls["silver"] = pd.to_numeric(metalls["silver"])
        metalls["platinum"] = pd.to_numeric(metalls["platinum"].str.replace(" ", ""))
        metalls["palladium"] = pd.to_numeric(metalls["palladium"].str.replace(" ", ""))

        metalls = metalls.sort_values("date").reset_index(drop=True)

        with pd.ExcelWriter(
            "./data/cb_metalls.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            metalls.to_excel(writer, sheet_name="cb_metalls")

        print("CB_metalls parsing is DONE!")
        display(metalls.tail(20))

    except Exception as error:
        print(error)


In [8]:
################################################################
#########################   NBK ################################
################################################################

async def nbk_tenge_async():
    # Realy unrelieable source, mb it would be better off with using ms query inside the file
    try:
        year = date.today().year

        upper_bound = f'01.01.2022'
        lower_bound = f'31.12.{year}'

        url = f'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut\
            /report?rates%5B%5D=5&beginDate={upper_bound}&endDate={lower_bound}'

        counter = 0

        while counter <= 6:
            try:
                with time_limit(15):
                    page = requests.get(url=url)
                    break

            except TimeoutException:
                # print("NBK_tenge timed out! Another attempt")
                counter += 1

        temp_df = pd.read_html(page.text)
        df = temp_df[0]
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])  # , dayfirst=True)
        df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

        with pd.ExcelWriter(
                './data/nbk_tenge.xlsx') as writer:
            df.to_excel(writer, sheet_name='tenge')

        print('NBK_tenge parsing is DONE!')
        display(df.tail(10))

        return df
    
    except Exception as error:
        print(error)

In [9]:
################################################################
##########################  SHMET ##############################
################################################################

async def shmet_optimized_async():
    url = "https://en.shmet.com/api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0"
    
    try:
        responce = requests.get(url)

        day_df = pd.DataFrame(responce.json()["data"])
        day_df["date"] = date.today()
        cooper_row = day_df[day_df["name"].str.contains("cu", case=False)]

        result = cooper_row[["date", "middle", "unit"]]
        result["date"] = pd.to_datetime(result["date"])
        result = result.rename(columns={"middle": "price"})

        hist_data = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)

        new_df = pd.concat([result, hist_data], axis=0).reset_index(drop=True)
        new_df.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
            "./data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            new_df.to_excel(writer, sheet_name="SHMET")

        final = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
        final.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
            "./data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            final.to_excel(writer, sheet_name="SHMET")

        print("SHMET is done!!!")
        display(final.head(10))
    
    except Exception as error:
        print(error)


In [10]:
################################################################
################## NEW_WESTMETALL###############################
################################################################


async def new_westmetall_async():
    async def get_data(metall, col_name):
        url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash"
        response = requests.get(url=url)

        df = pd.read_html(response.text)[0][:30]
        data = df.iloc[:, :2]

        data = data.query("date != 'date'")
        data["date"] = pd.to_datetime(data["date"])
        data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
        data.rename(columns={data.columns[1]: col_name}, inplace=True)

        return data

    try:
        al = await get_data(metall="Al", col_name="aluminium")
        cu = await get_data(metall="Cu", col_name="copper")
        pb = await get_data(metall="Pb", col_name="lead")
        ni = await get_data(metall="Ni", col_name="nickel")
        zn = await get_data(metall="Zn", col_name="zink")
        tn = await get_data(metall="Sn", col_name="tin")

        result = (
            pd.merge(al, cu, on="date", how="left")
            .merge(pb, on="date", how="left")
            .merge(ni, on="date", how="left")
            .merge(zn, on="date", how="left")
            .merge(tn, on="date", how="left")
        )

        old_data = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

        final_data = pd.concat([old_data, result], axis=0)

        final_data.drop_duplicates(subset="date", inplace=True)

        final_data.sort_values(by="date", inplace=True)

        final_data.reset_index(inplace=True, drop=True)

        with pd.ExcelWriter(
            "./data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            final_data.to_excel(writer, sheet_name="LME_westmetall")

        print("WESTMETALL is done!!!")
        display(final_data.tail(10))

    except Exception as error:
        print(error)


In [11]:
if __name__ == "__main__":

    await asyncio.gather(
        kitco_parser_async(),
        lbma_prescious_async(),
        lme_selenium_async(),
        new_westmetall_async(),
        cb_currency(),
        cb_metalls(),
        nbk_tenge_async(),
        shmet_optimized_async(),
    )
    
    # time.sleep(10)
    # Аплоад
    #writer(time_out=7)

KITKO_main is done!!!


,Date,Gold,Silver,Platinum,Palladium
469,2024-11-08,2691.15,31.55,987.0,1013.0
470,2024-11-11,2624.75,31.32,973.0,990.0
471,2024-11-12,2606.85,30.41,957.0,967.0
472,2024-11-13,2598.75,30.91,948.0,946.0
473,2024-11-14,2567.30,29.99,934.0,923.0
474,2024-11-15,2571.80,30.66,944.0,951.0
475,2024-11-18,2606.85,30.74,959.0,991.0
476,2024-11-19,2623.20,31.29,970.0,1030.0
477,2024-11-20,2640.55,30.87,970.0,1033.0
482,2024-11-21,2665.30,31.07,960.0,1028.0


LBMA is done!!!


,Date,Gold,Silver,Platinum,Palladium
14395,2024-11-08,2691.15,31.550,987.0,1013.0
14396,2024-11-11,2624.75,31.315,973.0,990.0
14397,2024-11-12,2606.85,30.405,957.0,967.0
14398,2024-11-13,2598.75,30.910,948.0,946.0
14399,2024-11-14,2567.30,29.990,934.0,923.0
14400,2024-11-15,2571.80,30.660,944.0,951.0
14401,2024-11-18,2606.85,30.740,959.0,991.0
14402,2024-11-19,2623.20,31.285,970.0,1030.0
14403,2024-11-20,2640.55,30.870,970.0,1033.0
14407,2024-11-21,2665.30,31.065,960.0,1028.0


LME_main is done!!!


...

,date,gold,silver,platinum,palladium
448,2024-10-26,8490.71,106.09,3216.65,3527.43
449,2024-10-29,8538.55,103.63,3185.41,3676.19
450,2024-10-30,8579.38,104.47,3232.36,3761.18
451,2024-10-31,8640.65,106.54,3276.34,3813.04
452,2024-11-01,8664.92,106.15,3181.73,3605.96
453,2024-11-02,8565.47,105.23,3117.11,3524.37
454,2024-11-03,8606.95,102.87,3148.84,3531.47
455,2024-11-06,8646.26,103.53,3143.12,3417.39
456,2024-11-07,8660.86,103.11,3161.12,3448.49
457,2024-11-08,8387.89,100.27,3067.97,3272.92


NBK_tenge parsing is DONE!


,date,Числовое значение,ДОЛЛАР США
1047,2024-11-13,1,494.95
1048,2024-11-14,1,492.17
1049,2024-11-15,1,493.85
1050,2024-11-16,1,495.95
1051,2024-11-17,1,495.95
1052,2024-11-18,1,495.95
1053,2024-11-19,1,497.89
1054,2024-11-20,1,497.26
1055,2024-11-21,1,495.20
1056,2024-11-22,1,494.87


SHMET is done!!!


,date,price,unit
0,2024-11-22,74425,Yuan/MT
1,2024-11-21,74690,Yuan/MT
2,2024-11-20,74220,Yuan/MT
3,2024-11-19,73820,Yuan/MT
4,2024-11-18,74050,Yuan/MT
5,2024-11-16,74050,Yuan/MT
6,2024-11-15,73400,Yuan/MT
7,2024-11-14,74740,Yuan/MT
8,2024-11-13,75930,Yuan/MT
9,2024-11-12,76490,Yuan/MT


In [ ]:
# writer()

Database uploaded on yandex.disk
Ресурс "disk:/macro_db/database.xlsx" уже существует. (Resource "disk:/macro_db/database.xlsx" already exists. / DiskResourceAlreadyExistsError)
LME_non_ferrous uploaded!
KITCO uploaded!
LBMA uploaded!
nbk_tenge uploaded!
SHMET uploaded!
LME_westmetall uploaded!
USD uploaded!
EUR uploaded!
Australian_Dollar uploaded!
China_Yuan uploaded!
British_Pound uploaded!
Kazakhstan_Tenge uploaded!
Japanese_Yen uploaded!
Swiss_Franc uploaded!
cb_metalls uploaded!
Uploading whole file to Google Sheets completed
